<a href="https://colab.research.google.com/github/dude123studios/TransformerLearning/blob/main/Pytorch_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
from torch.utils.data import DataLoader
from torch import nn
from torchvision import datasets, transforms
import os

In [9]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=transforms.ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=transforms.ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
device = 'cuda' if torch.cuda.is_available() else  'cpu'
print(f'Using {device} device ')

Using cpu device 


In [4]:
class NueralNetwork(nn.Module):
    def __init__(self):
        super(NueralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(nn.Linear(28 * 28, 512),
                                               nn.ReLU(), nn.Linear(512, 512),
                                               nn.ReLU(), nn.Linear(512, 10))

    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [5]:
model = NueralNetwork().to(device)
print(model)

NueralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
x = torch.rand((1, 28, 28), device=device)
logits = model(x)
pred_prob = nn.Softmax(dim=1)(logits)
y_pred = pred_prob.argmax(1)
print(f'Predicted class {y_pred}')

Predicted class tensor([3])


In [15]:
learning_rate = 1e-3
batch_size = 64
epochs = 10

In [13]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):

        #Forward Pass
        pred = model(X)
        loss = loss_fn(pred, y)

        #Backward Pass
        optimizer.zero_grad() #Reset optimizer's computed grads to 0
        loss.backward() #Compute all gradients
        optimizer.step() #Update weights with each of their newly computed gradients

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [14]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

In [16]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.306278  [    0/60000]
loss: 2.296587  [ 6400/60000]
loss: 2.274753  [12800/60000]
loss: 2.264477  [19200/60000]
loss: 2.243316  [25600/60000]
loss: 2.224376  [32000/60000]
loss: 2.222136  [38400/60000]
loss: 2.195870  [44800/60000]
loss: 2.195055  [51200/60000]
loss: 2.156963  [57600/60000]
Epoch 2
-------------------------------
loss: 2.164903  [    0/60000]
loss: 2.156428  [ 6400/60000]
loss: 2.094105  [12800/60000]
loss: 2.107533  [19200/60000]
loss: 2.043602  [25600/60000]
loss: 1.997468  [32000/60000]
loss: 2.018254  [38400/60000]
loss: 1.943678  [44800/60000]
loss: 1.954789  [51200/60000]
loss: 1.875761  [57600/60000]
Epoch 3
-------------------------------
loss: 1.907375  [    0/60000]
loss: 1.879389  [ 6400/60000]
loss: 1.756700  [12800/60000]
loss: 1.800603  [19200/60000]
loss: 1.679145  [25600/60000]
loss: 1.642569  [32000/60000]
loss: 1.660160  [38400/60000]
loss: 1.567812  [44800/60000]
loss: 1.596884  [51200/60000]
loss: 1.49